In [1]:
import pandas as pd
path = r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\new_folder\2_ cathay-ins(2024_08_05-2024_09_01).xlsx"
data = pd.ExcelFile(path)
df = data.parse()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   关键词            1000 non-null   object 
 1   点击量            1000 non-null   object 
 2   Traffic        732 non-null    object 
 3   变动             1000 non-null   object 
 4   28 天的体量        1000 non-null   int64  
 5   流量变化           1000 non-null   object 
 6   桌面端份额          1000 non-null   object 
 7   移动端份额          1000 non-null   object 
 8   KD             557 non-null    float64
 9   意图             581 non-null    object 
 10  平均体量           1000 non-null   object 
 11  CPC            1000 non-null   object 
 12  零点击            741 non-null    float64
 13  热门网址           1000 non-null   object 
 14  SERP Features  781 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 117.3+ KB


In [2]:
df.columns

Index(['关键词', '点击量', 'Traffic ', '变动', '28 天的体量', '流量变化', '桌面端份额', '移动端份额',
       'KD', '意图', '平均体量', 'CPC', '零点击', '热门网址', 'SERP Features'],
      dtype='object')

In [3]:
# 只使用三欄位
three_df_old = df[['关键词', '点击量', '热门网址']]
three_df = three_df_old.rename(columns={'关键词':"Keyword", '点击量':"Clicks", '热门网址':"Website"})
three_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Keyword  1000 non-null   object
 1   Clicks   1000 non-null   object
 2   Website  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


# 清洗

In [4]:
# not int
non_int_row = three_df.loc[three_df['Clicks'].apply(lambda x: not isinstance(x, int))]
non_int_row.count()


Keyword    586
Clicks     586
Website    586
dtype: int64

In [5]:
# type of string
unique_values = non_int_row['Clicks'].unique() # return a Numpyarray
print(unique_values)

['<50' '0']


In [6]:
# counts of "<50"
str_row1 = non_int_row.loc[non_int_row['Clicks'].apply(lambda x: x == "<50")]
str_row1.count()


Keyword    318
Clicks     318
Website    318
dtype: int64

In [7]:
# counts of "0"
str_row2 = non_int_row.loc[non_int_row['Clicks'].apply(lambda x: x == "0")]
str_row2.count()

Keyword    268
Clicks     268
Website    268
dtype: int64

In [8]:
# string -> 0
three_df.loc[:, 'Clicks'] = three_df['Clicks'].apply(lambda x: 0 if isinstance(x, str) else x)
three_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Keyword  1000 non-null   object
 1   Clicks   1000 non-null   object
 2   Website  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


# 分組

In [9]:
# 包含國泰 contain_df
contain_df = three_df.loc[three_df['Keyword'].str.contains('國泰', na=False)]
contain_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211 entries, 0 to 994
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Keyword  211 non-null    object
 1   Clicks   211 non-null    object
 2   Website  211 non-null    object
dtypes: object(3)
memory usage: 6.6+ KB


In [10]:
# 不包含國泰 not_contain_df
not_contain_df = three_df.loc[~three_df['Keyword'].str.contains('國泰', na=False)]
not_contain_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 789 entries, 1 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Keyword  789 non-null    object
 1   Clicks   789 non-null    object
 2   Website  789 non-null    object
dtypes: object(3)
memory usage: 24.7+ KB


# 聚合

In [11]:
# 包含國泰
group_df1 = contain_df.groupby("Website", as_index=False).agg({ # as_index=False: Website not as a new index
    "Clicks":"sum",
    "Keyword":lambda x: set(x)
})  

contain_sorted_df = group_df1.sort_values(by="Clicks", ascending=False)
contain_sorted_df.head(5)

,Website,Clicks,Keyword
45,cathay-ins.com.tw/cathayins/personal,31390,"{國泰, 國泰產檢, 國泰產險網站, 國泰世紀產物, 國泰車險, 國泰 車險, 國泰產物線上..."
8,cathay-ins.com.tw/INSEBWeb/BOBE/travel/travel_...,8620,"{國泰旅遊不便險, 國泰人壽 不便險, 國泰 旅平險, 國泰不便險, 國泰產險 旅平險, 國..."
29,cathay-ins.com.tw/INSPFWeb/servlet/HttpDispatc...,4520,"{國泰人壽客服電話, 國泰產險 客服信箱, 國泰人壽客服, 國泰產險客服, 國泰信用卡 客服..."
11,cathay-ins.com.tw/INSEBWeb/servlet/HttpDispatc...,2510,"{國泰產 機車, 國泰強制險, 機車強制險 國泰, 國泰機車三責, 國泰 機車保險, 國泰機..."
82,smartpay.cathay-ins.com.tw,2240,"{國泰保險查詢, 國泰保單號碼查詢, 國泰保單查詢, 國泰產險 登入, 國泰產險保單查詢, ..."


In [12]:
contain_sorted_df.reset_index(drop=True, inplace=True)
contain_sorted_df.head(2)

,Website,Clicks,Keyword
0,cathay-ins.com.tw/cathayins/personal,31390,"{國泰, 國泰產檢, 國泰產險網站, 國泰世紀產物, 國泰車險, 國泰 車險, 國泰產物線上..."
1,cathay-ins.com.tw/INSEBWeb/BOBE/travel/travel_...,8620,"{國泰旅遊不便險, 國泰人壽 不便險, 國泰 旅平險, 國泰不便險, 國泰產險 旅平險, 國..."


In [13]:
# 不包含國泰
group_df2 = not_contain_df.groupby("Website", as_index=False).agg({ # as_index=False: Website not as a new index
    "Clicks":"sum",
    "Keyword":lambda x: set(x)
})  

not_contain_sorted_df = group_df2.sort_values(by="Clicks", ascending=False)
not_contain_sorted_df.head(5)

,Website,Clicks,Keyword
19,cathay-ins.com.tw/INSEBWeb/BOBE/travel/travel_...,26450,"{旅平險, 美國旅遊險 ptt, 15歲以下旅平險線上投保, 不便險 多家投保, 旅行平安險..."
122,cathay-ins.com.tw/cathayins/personal/motorcycl...,7400,"{機車強制險, 強制險 機車, 機車強制責任險, 機車強制險一年多少錢, 機車強制險費用, ..."
70,cathay-ins.com.tw/cathayins/personal/car/liabi...,5670,"{汽車第三責任險費用, 汽車第三責任險, 32撞是什麼, 第三人責任保險, 強制險有賠財損嗎..."
102,cathay-ins.com.tw/cathayins/personal/knowledge...,3370,"{機車 牌照稅, 汽車牌照, 牌照, 燃料稅過期一天, 牌照稅級距, 監理站牌照稅查詢, 汽..."
68,cathay-ins.com.tw/cathayins/personal/car,2800,"{新車 保險 生效日, 汽車 保險, 汽車保險推薦, 汽車保險 推薦, 車子保險, 汽車保險..."


In [14]:
# reset index
not_contain_sorted_df.reset_index(drop=True, inplace=True) # drop=True: drop old index
not_contain_sorted_df.head(2)

,Website,Clicks,Keyword
0,cathay-ins.com.tw/INSEBWeb/BOBE/travel/travel_...,26450,"{旅平險, 美國旅遊險 ptt, 15歲以下旅平險線上投保, 不便險 多家投保, 旅行平安險..."
1,cathay-ins.com.tw/cathayins/personal/motorcycl...,7400,"{機車強制險, 強制險 機車, 機車強制責任險, 機車強制險一年多少錢, 機車強制險費用, ..."


# 總整理

In [15]:
# Sheet1
organic_all_data = pd.DataFrame()
organic_all_data["國泰_有國泰"] = contain_sorted_df["Keyword"].head(5)
organic_all_data["國泰_沒國泰"] = not_contain_sorted_df["Keyword"].head(5)
with pd.ExcelWriter("organic_all_data.xlsx", mode='w') as writer:
    organic_all_data.to_excel(writer, sheet_name="Sheet1", index=False)

organic_all_data

,國泰_有國泰,國泰_沒國泰
0,"{國泰, 國泰產檢, 國泰產險網站, 國泰世紀產物, 國泰車險, 國泰 車險, 國泰產物線上...","{旅平險, 美國旅遊險 ptt, 15歲以下旅平險線上投保, 不便險 多家投保, 旅行平安險..."
1,"{國泰旅遊不便險, 國泰人壽 不便險, 國泰 旅平險, 國泰不便險, 國泰產險 旅平險, 國...","{機車強制險, 強制險 機車, 機車強制責任險, 機車強制險一年多少錢, 機車強制險費用, ..."
2,"{國泰人壽客服電話, 國泰產險 客服信箱, 國泰人壽客服, 國泰產險客服, 國泰信用卡 客服...","{汽車第三責任險費用, 汽車第三責任險, 32撞是什麼, 第三人責任保險, 強制險有賠財損嗎..."
3,"{國泰產 機車, 國泰強制險, 機車強制險 國泰, 國泰機車三責, 國泰 機車保險, 國泰機...","{機車 牌照稅, 汽車牌照, 牌照, 燃料稅過期一天, 牌照稅級距, 監理站牌照稅查詢, 汽..."
4,"{國泰保險查詢, 國泰保單號碼查詢, 國泰保單查詢, 國泰產險 登入, 國泰產險保單查詢, ...","{新車 保險 生效日, 汽車 保險, 汽車保險推薦, 汽車保險 推薦, 車子保險, 汽車保險..."


In [16]:
# 新sheet "國泰_有國泰"
file = r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\organic_all_data.xlsx"
with pd.ExcelWriter(file, mode='a', if_sheet_exists="overlay") as writer:
    contain_sorted_df.to_excel(writer, sheet_name='國泰_有國泰', index=False)

In [17]:
with pd.ExcelWriter(file, mode='a', if_sheet_exists="overlay") as writer:
    not_contain_sorted_df.to_excel(writer, sheet_name='國泰_沒國泰', index=False)# 新sheet "國泰_沒國泰"